In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('sentiment_score.csv')

In [3]:
df.columns

Index(['Unnamed: 0', 'quarter_index', 'sentiment_score', 'number_posts',
       'normed_value', 'sales'],
      dtype='object')

In [4]:
y_val = np.array(np.log(df.loc[:,'sales']).diff().iloc[2:38,])

In [5]:
x_val_lag = np.reshape(np.array(np.log(df.loc[:,'sales']).diff().iloc[1:37,]), (-1,1))
x_sent_lag = np.reshape(np.array(df.loc[1:36,'normed_value']), (-1,1))
x_npost_lag = np.reshape(np.array(df.loc[1:36,'number_posts']), (-1,1))

In [6]:
final_df_val = np.concatenate((x_val_lag, x_sent_lag, x_npost_lag), axis = 1)

In [10]:
from sklearn.gaussian_process import GaussianProcessRegressor
import sklearn.gaussian_process as gp
from sklearn import preprocessing

predict_result = list()
real_result = list()

num_train = 15
num_var = final_df_val.shape[1]

kernel1 = gp.kernels.Matern(nu=2.5)
kernel2 = gp.kernels.RationalQuadratic()
kernel3 = gp.kernels.RBF()
kernel4 = gp.kernels.WhiteKernel()

for i in range(len(y_val) - num_train - 1):
    temp_df = final_df_val[0:num_train + i + 1,]
    temp_df = preprocessing.scale(temp_df)
    
    each_train_X = np.reshape(temp_df[0:i+num_train,], (-1,num_var))
    each_train_Y = y_val[0+1:i+num_train+1]
    each_train_Y[each_train_Y == 0] = 1
    
    #lmnn.fit(each_train_X, each_train_Y)
    #each_train_X = lmnn.transform(each_train_X)
    
    gpr_test = GaussianProcessRegressor(n_restarts_optimizer = 10,kernel = kernel1 + kernel2 + kernel3 + kernel4)
    gpr_test.fit(each_train_X, each_train_Y)
    
    #sam_weight = weight_generator(each_train_X.shape[0])
    
    each_test_X = np.reshape(temp_df[num_train + i,], (-1,num_var))
    each_test_Y = y_val[num_train + i + 1]
    
    #each_test_X = lmnn.transform(each_test_X)
    
    gpr_test.predict(each_test_X)
    
    predict_result.append(gpr_test.predict(each_test_X)[0])
    if isinstance(each_test_Y, list):
        real_result.append(each_test_Y[0])
    else:
        real_result.append(each_test_Y)
    
    if i % 10 == 0:
        print(i)
        print(np.sqrt(np.mean(np.square(np.array(predict_result) - np.array(real_result)))))

0
0.11333725156240615
10
0.20364168714527164


In [11]:
np.sqrt(np.mean(np.square(np.array(predict_result) - np.array(real_result))))

0.24328899242565846

In [16]:
gpr_test.score(each_train_X, each_train_Y)

0.9999999789650864

In [17]:
gpr_test.get_params()

{'alpha': 1e-10,
 'copy_X_train': True,
 'kernel__k1': Matern(length_scale=1, nu=2.5) + RationalQuadratic(alpha=1, length_scale=1) + RBF(length_scale=1),
 'kernel__k2': WhiteKernel(noise_level=1),
 'kernel__k1__k1': Matern(length_scale=1, nu=2.5) + RationalQuadratic(alpha=1, length_scale=1),
 'kernel__k1__k2': RBF(length_scale=1),
 'kernel__k1__k1__k1': Matern(length_scale=1, nu=2.5),
 'kernel__k1__k1__k2': RationalQuadratic(alpha=1, length_scale=1),
 'kernel__k1__k1__k1__length_scale': 1.0,
 'kernel__k1__k1__k1__length_scale_bounds': (1e-05, 100000.0),
 'kernel__k1__k1__k1__nu': 2.5,
 'kernel__k1__k1__k2__length_scale': 1.0,
 'kernel__k1__k1__k2__alpha': 1.0,
 'kernel__k1__k1__k2__length_scale_bounds': (1e-05, 100000.0),
 'kernel__k1__k1__k2__alpha_bounds': (1e-05, 100000.0),
 'kernel__k1__k2__length_scale': 1.0,
 'kernel__k1__k2__length_scale_bounds': (1e-05, 100000.0),
 'kernel__k2__noise_level': 1.0,
 'kernel__k2__noise_level_bounds': (1e-05, 100000.0),
 'kernel': Matern(length_sca